# Medical Cost Dataset
This Jupyter notebook contains the code used during OUAIxHacklahoma workshop for Fall 2021 (December 1st, 2021). Credits given to _miri choi_ from Kaggle for collecting and organizing the provided ["Medical Cost Dataset"](https://www.kaggle.com/mirichoi0218/insurance).

Objectives
--------------

This jupyter notebook implements a solution to best categorize charges incurred to a person for insurance. This task is exclusively of regression, and the insurance cost is in dollars.

In [ ]:
import numpy as np
import pandas as pd

Data Obtention
============

The Medical Cost dataset is provided in the datasets folder. We use Pandas's `read_csv` function. More formats are supported; read more in pandas [documentation](https://pandas.pydata.org/docs/reference/io.html).

In [ ]:
insurance = pd.read_csv('datasets/insurance.csv')

Data Discovery
============

__It is important for us to gain an understanding of our data before performing any kind of learning or analysis.__ It is important to note that by "Discovery" we mean understanding what attributes and data types the dataset includes. This also includes identifying usefull and useless data attributes (such as dates or indexes). At this stage we want to focus in geting a grasp on what needs to be done next to achieve our goals.

### Look at the dataset's attributes.

In [ ]:
insurance.columns

### Look at the 7 first and last entries.

In [ ]:
insurance.head(7)

In [ ]:
insurance.tail(7)

### See what kind of data the dataset contains.
Note that `sex`, `smoker`, and `region` attributes are shown as `object` Dtype. When data is first imported into sklearn, strings are represented as object types. Oftentimes, this data is categorical, and because we have imported a CSV data file we can rest assured that the columns will be categorical. We can further prove this by taking a look at the values that exist within the suspected categorical attributes.

In [ ]:
insurance.info()

In [ ]:
insurance['region'].value_counts()

## Dataset Statistics

The `describe()` function provided by Pandas allows us to get a glimpse into the statistical relationships going on in the dataset. This information can be useful to gain a better understanding of the dataset itself.

In [ ]:
insurance.describe()

Note that only __4__ attributes are displayed. This is due to the fact that the describe function only looks at numerical data. Other types of data are ignored.

#### Plotting the dataset

We can further gain knowledge by ploting the dataset. Pay close attention to the distribution! The `hist()` function allows us to plot a historgram. These are very useful when dealing with data that may have different statistical distributions. The `bins` parameter allows us to choose how many "rectangles" to include in a single histogram. The `figsize` allows us to determine the size of the histogram (y, x).

In [ ]:
insurance.hist(bins=50, figsize=(20, 15))

# Dataset Preparation

Once we have gained a better understanding of the kind of data that we are dealing with, it is important to avoid making assumptions on the dataset. This is because certain attributes may not influence the final prediction as we may think. This is crucial when seeking the best machine learning model.

When working with machine learning, it is very important for us to prepare the dataset for training. This is accomplished by first separating the data into two parts: _Training_ and _Testing_. Doing this will allow us to maintain a neutral position on the dataset and avoid outside bias from interfering with the learning process. Additionally, we will be using the testing dataset after we perform learning in order for us to test how well our model does with unseen data. At the end of the day, we care about a model that can do well with unseen data rather than just the data that it has learned already.

`StratifiedShuffleSplit` is a commonly-used splitting technique to separate the training from the testing sub-datasets. It uses a categorical attribute to keep the best representation among the datasets. In data science, the __80/20__ splitting rule is the most used splitting distribution. We keep 80% for our training data and 20% for our testing data.

### How do we choose which attribute to split on?

Expert advise is best when deciding what attribute to use. In general, it is a good idea to split your dataset using as basis an attribute that may be very important for machine learning; thus, it is benefitial to maintain the best representation of the data possible.

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

attrib = 'age'
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_i, test_i in split.split(insurance, insurance[attrib]):
    strat_train_set = insurance.loc[train_i]
    strat_test_set = insurance.loc[test_i]

In [ ]:
strat_train_set.head()

In [ ]:
strat_test_set.head()

We can apreciate the dataset distribution

In [ ]:
# Train dataset
strat_train_set[attrib].value_counts().hist(bins=100, figsize=(10, 5))

In [ ]:
# Test dataset
strat_test_set[attrib].value_counts().hist(bins=100, figsize=(10, 5))

In [ ]:
# The distribution is kept if the graphs of the other datasets resemble this one.
insurance[attrib].value_counts().hist(bins=100, figsize=(10, 5))

From now on, __only use the training dataset__ to perform any further data analysis.

## Correlations

Given that in data science we are attempting to find correlations within the data that could give the machine learning model better insights, it is a good idea to check what attributes may be the most useful for this. Note that this will only work with numerical data, thus, we may not be able to use simple Pearson correlation on attributes that are categorical or `object` type.

In [ ]:
# Create a correlation matrix
corr_matrix = strat_train_set.corr()
corr_matrix['charges'].sort_values(ascending=False)

It seems to be that age is the best predictor of insurance cost. Note that this correlation value is very low. Since a significant number of attributes are `object` type it is worth the attempt to try and convert those attributes to a numerical encoding to check if there are better correlations. To do this, we can use sklearn's OrdinalEncoder to convert categorical strings into numerical form.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

cat_cols = ['sex', 'smoker', 'region']
oe = OrdinalEncoder()
strat_train_set[cat_cols] = oe.fit_transform(strat_train_set[cat_cols])

strat_train_set.describe()

In [ ]:
# Create a correlation matrix and check correlations again
corr_matrix = strat_train_set.corr()
corr_matrix['charges'].sort_values(ascending=False)

Note how being a smoker gives a positive correlation. This correlation is also much better than any other attribute.

### Scatter Matrix

We can further visualize any correlations among the data attributes by using a correlation matrix. This can help us better observe the above correlations.

In [ ]:
insurance.columns

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges']
scatter_matrix(strat_train_set[attributes], figsize=(12, 8))

# Attribute Engineering

Often, it is a good idea to try and combine attributes to further enrich the data of information that may not be there by the individual attributes. This is particularily used for numerical attributes. __Avoid using the label when combining features.__ This is important since the learning model may get information by the label indirectly through the combined attribute. Further, the label will not be available when we actually deploy the model (since we are trying to predict the label to begin with!)

The most promising attributes are `bmi` and `age`.

In [ ]:
# Create dummy dataset so that we can play more safely with our dataset.
attr_eng_train_set = strat_train_set.copy()
attr_eng_train_set['bmi.per.year'] = attr_eng_train_set['bmi']/attr_eng_train_set['age']
attr_eng_train_set['bmi.year'] = attr_eng_train_set['bmi']*attr_eng_train_set['age']

In [ ]:
corr_matrix = attr_eng_train_set.corr()
corr_matrix['charges'].sort_values(ascending=False)

Note that `bmi.per.year` only has a slight negative correlation with charges. `bmi.year` on the other hand adds a better correlation than most other attributes. Because of this, we will add this new attribute combination to the original dataset so that when we resplit the dataset for learning, we keep this new attribute.

In [ ]:
insurance['bmi.year'] = insurance['bmi']*insurance['age']

With our newly conceived knowledge about the dataset, we can resplit the dataset, now using a better estimate for splitting.

In [ ]:
attrib = 'region'
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_i, test_i in split.split(insurance, insurance[attrib]):
    strat_train_set = insurance.loc[train_i]
    strat_test_set = insurance.loc[test_i]

strat_train_set.reset_index(inplace=True, drop=True)
strat_test_set.reset_index(inplace=True, drop=True)

# Dataset Preparation

Commonly, datasets have missing or invalid data. These data samples usually show as NaN values when looking at the data. In our case we do not have such an issue. This dataset has been precleaned by the dataset provided; thus, no further need for dataset cleaning is needed. In our other example with the Adult dataset we will encounter several missing values. Further discussion on how to work with incomplete data is given in the jupyter notebook for that dataset.

## Dealing with Categorical Data

Often, machine learning models need special representation of categorical attributes. The reason for this is on their training algorithms that use optimization strategies that require numerical values. Often, it is enough for us to replace categorical values with integers. The categorical values we will be dealing with are:`sex`, `smoker`, `region`.

In [ ]:
strat_train_set.head()

### Encoding Categorical Attributes

There are seberal common ways to deal with categorical attributes. The first one is to encode categories using numbers for each respective category. In this manner, region northwest could be given a value of 0, northeast a value of 1, southwest a value of 2, and southeast a value of 3. This can be achieved by using eithwe `OrdinalEncoder` or `LabelEncoder` from scikit-learn.

Often, a third option is used called `OneHotEncoder`. One-hot encoding uses several columns to encode categorical data in bit-string format. for example, encoding the region attribute would use 4 columns. Each colum would represent whether a sample is of one type or another. For example, if the first column has a 1 and all other columns have a 0, then, we could say that the region is northwest. If the third column has a 1 and all other columns a value of 0, then we could say that the region for that sample is southwest. 

The benefit of using one-hot encoding is that by not placing a numerical value on the categories, we do not create a relationship among them where there may not exist one. For example, giving a higher value to southwest compare to northeast in region attribute could cause the machine learning algorithm to pick an inexisting relationship among attributes. One-hot encoding alleviates this problem.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
cat_attrs = ['sex', 'region', 'smoker']

ohe = OneHotEncoder()
ohe.fit(insurance[cat_attrs])
ohe_train = pd.DataFrame(ohe.transform(strat_train_set[cat_attrs]).toarray())
ohe_test = pd.DataFrame(ohe.transform(strat_test_set[cat_attrs]).toarray())

# Drop categorical data
strat_train_set.drop(cat_attrs, axis=1, inplace=True)
strat_test_set.drop(cat_attrs, axis=1, inplace=True)

# Replace with one-hot encoded data
strat_train_set = strat_train_set.join(ohe_train)
strat_test_set = strat_test_set.join(ohe_test)

# Kepp column names as string
strat_train_set.columns = strat_train_set.columns.astype(str)
strat_test_set.columns = strat_test_set.columns.astype(str)


In [ ]:
strat_train_set.head()

In [ ]:
strat_test_set.head()

## Feature Scalling

Sometimes, data comes in varying degrees of scales. For example, our dataset attribute `bmi.year` is widely distributed. This is evident when we look at the `describe()` function of the dataset and note the difference between max and min statistics. These large numbers can make machine learning models take longer than expected to learn a solution. to make learning easier, it is good practice to "standarize" or "noramlize" our data. There are two main ways by which this can be achieved:

- min-max scaling:
  - We use the difference between the minimum and maximum values in the dataset to keep data between 0 and 1.
- standarization:
  - We use the mean to "de-mean" the data. Then, we divide the data by the standard deviation to scale values to unit variances.

Note that the main difference is that min-max gives values between 0 and 1 and standarization does not.

There is no particular rule to when to use each type of scaling. It is a good idea to try both and compare how much each improves data prediction.

In [ ]:
insurance.describe()

As a side note, it can be also a good idea to scale the charges attribute. This is common practice in machine learning, but it requires reversing this scaling to obtain the true estimate. Because we attempt to keep this workshop as simple as possible we will avoid this step for now. Further, this step is usually taken when the value is trully large such as in the quatrillions and above.

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler


to_scale = ['bmi.year']
# We use two scalers to isolate both datasets
ss_train = StandardScaler()
ss_test = StandardScaler()
# mm_train = MinMaxScaler()
# mm_test = MinMaxScaler()

strat_train_set[to_scale] = ss_train.fit_transform(strat_train_set[to_scale])
strat_test_set[to_scale] = ss_test.fit_transform(strat_test_set[to_scale])

In [ ]:
strat_train_set.describe()

# Model Selection

There is a plethora of different machine learning models. Depending on the application, some models may be better than others. In this case, we are dealing with a classification problem. Because of this, some recomended models to try are:

- Ordinary Least Squares
- Ridge Regression
- Lasso
- Elastic Net
- Stochastic Gradient Descent
- Multi-Layer Perceptron

For training, we remove the labels from the training data. Leaving them in the dataset will be reminiscent to giving test answers to a class. It is common practice to use an X variable for the de-labeled data and y for the labels.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor

model = RandomForestRegressor()

# Remove labels.
X = strat_train_set.drop(['charges'], axis=1)
# Put labels in different variable
y = strat_train_set['charges']

# Here it is where the magic happend
model.fit(X, y)

In order for us to distinguish among models, it is a good idea to pick an accuracy estimator. There are many accuracy estimators that are used in different circumstances. Some of the most used are root mean squared error (RMSE) and mean absolute error (MAE). We highly recomend you read about other accuracy estimators for these estimators have special applications for different goals in data science.

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

train_pred = model.predict(X)

test_X = strat_test_set.drop(['charges'], axis=1)
test_pred = model.predict(test_X)
test_y = strat_test_set['charges']

train_rmse = np.sqrt(mean_squared_error(y, train_pred))
test_rmse = np.sqrt(mean_squared_error(test_y, test_pred))

train_mae = mean_absolute_error(y, train_pred)
test_mae = mean_absolute_error(test_y, test_pred)


In [ ]:
print(f"Training Score: {train_rmse} | Testing Score: {test_rmse}")

In [ ]:
print(f"Training Score: {train_mae} | Testing Score: {test_mae}")

As an interesting observation, testing a linear regressor produces a score that is worse for training dataset than testing dataset. This is because the the model may not be powerful enough for learning the relationships. Testing with a RandomForestRegressor produces a much better score.

In [ ]:
test_pred[:20]

In [ ]:
test_y[:20]

# Model Fine Tuning

As it can be appreciated, the scores obtained above are not phenomenal. The model definetely learns something, but the score reglects that when the model is put to the test, it still struggles. There can be many reasons for this, but one of the main reasons is that the model has not been fine tuned. Machine learning models come with pre-set attributes that may or may not be the best for a particular dataset. Because of this, it is a good idea to test different parameters. Often, testing new parameters will help the model perform better. Check out scikit-learn's documentation to find out about what parameters you can test to try.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Random Forest Regressor's
param_grid = [
    {
        'n_estimators': [50, 75, 100, 125, 150],
        'criterion': ['squared_error', 'absolute_error', 'poisson'],
    }
]
model = RandomForestRegressor()
grid_search = GridSearchCV(model, param_grid, cv=10, scoring='neg_mean_absolute_error', return_train_score=True, n_jobs=7)
grid_search.fit(X, y)

In [ ]:
best_model = grid_search.best_estimator_

train_pred = best_model.predict(X)

test_X = strat_test_set.drop(['charges'], axis=1)
test_pred = best_model.predict(test_X)
test_y = strat_test_set['charges']

train_rmse = np.sqrt(mean_squared_error(y, train_pred))
test_rmse = np.sqrt(mean_squared_error(test_y, test_pred))

train_mae = mean_absolute_error(y, train_pred)
test_mae = mean_absolute_error(test_y, test_pred)


In [ ]:
print(f"Training Score: {train_rmse} | Testing Score: {test_rmse}")

In [ ]:
print(f"Training Score: {train_mae} | Testing Score: {test_mae}")

Although slight, some improvement was made. Further fine tunning could make the score better.